# 🟦 miniROS — Notebook 4
## Occupancy Grid Mapping (SLAM Basics)

This notebook introduces **occupancy grids**, the foundation of SLAM and ROS mapping.

You will learn:
- How lidar data is integrated into a grid map
- How free and occupied cells are updated
- How ray tracing works
- How odometry affects map quality

---
## Step 1 — Download miniROS from GitHub


In [ ]:
!rm -rf miniROS
!git clone https://github.com/eder1234/miniROS.git
print("miniROS downloaded.")

## Step 2 — Import modules


In [ ]:
import sys
sys.path.append('/content/miniROS')

import numpy as np
import matplotlib.pyplot as plt

from core.simulator import MiniRosSimulator
from robots.diffdrive import DiffDriveRobot
from sensors.lidar import Lidar2D
from world.obstacles import CircleObstacle
from mapping.occupancy_grid import OccupancyGridMap
from core.messages import Pose2D, Twist
from core.node import Node

print("Imports OK.")

## Step 3 — Build world: robot, obstacles, lidar


In [ ]:
sim = MiniRosSimulator(dt=0.05)

robot = DiffDriveRobot(init_pose=Pose2D(0,0,0), radius=0.2, cmd_topic="/cmd_vel")
sim.add_robot(robot)

obstacles = [
    CircleObstacle(1.2, 0.3, 0.4),
    CircleObstacle(0.0, 1.5, 0.3),
    CircleObstacle(-1.0, 0.0, 0.5),
    CircleObstacle(1.5, -1.0, 0.4)
]

lidar = Lidar2D(robot, obstacles, num_beams=72, max_range=4.0)
sim.add_lidar(lidar)

print("World ready.")

## Step 4 — Define MappingNode

This node:
- Reads the `/scan` lidar topic
- Computes beam endpoints
- Performs ray tracing for free cells
- Marks impact cells as occupied


In [ ]:
class MappingNode(Node):
    def __init__(self):
        super().__init__("mapping")
        self.map = OccupancyGridMap(width=300, height=300, resolution=0.03)

    def step(self, dt):
        scan = self.read_topic("/scan")
        if scan is None:
            return

        x0 = self._sim.robots[0].pose.x
        y0 = self._sim.robots[0].pose.y
        yaw = self._sim.robots[0].pose.yaw

        for r, a in zip(scan.ranges, scan.angles):
            x1 = x0 + r * np.cos(yaw + a)
            y1 = y0 + r * np.sin(yaw + a)

            # Free space
            self.map.raytrace(x0, y0, x1, y1, steps=40)

            # Occupied
            if r < 3.9:  # near max
                self.map.set_occupied(x1, y1)

map_node = MappingNode()
sim.add_node(map_node)
print("MappingNode added.")

## Step 5 — Define simple exploration node
This moves the robot in circles to explore the environment.

In [ ]:
class ExploreNode(Node):
    def __init__(self):
        super().__init__("explore")

    def step(self, dt):
        t = self._sim.time % 3.0
        if t < 1.5:
            self.publish("/cmd_vel", Twist(0.4, 0.0))
        else:
            self.publish("/cmd_vel", Twist(0.0, 1.0))

sim.add_node(ExploreNode())
print("ExploreNode added.")

## Step 6 — Run simulation


In [ ]:
sim.run(20.0)
print("Mapping complete.")

## Step 7 — Visualize the Occupancy Grid


In [ ]:
grid = map_node.map.grid
plt.figure(figsize=(6,6))
plt.imshow(grid, cmap='gray_r', origin='upper')
plt.title("miniROS — Occupancy Grid Map")
plt.axis('off')
plt.show()

print("Unique grid values:", np.unique(grid))

---
# 🧪 Exercises
1. Add odometry noise to test map distortion.
2. Add more obstacles.
3. Change robot exploration behavior.
4. Use Bresenham algorithm for ray tracing.
5. Overlay robot trajectory on the map.

---
## End of Notebook 4
Next: **Notebook 5 — Navigation Pipeline**